In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D 
import seaborn as sns
from scipy.ndimage import gaussian_filter1d
import pickle as pkl
from mne import read_epochs

In [2]:
burst_df = pd.read_csv(r'C:\Users\qmoreau\Documents\Work\Beta_bursts\Bursts\burst_features.csv')

MemoryError: Unable to allocate 2.67 GiB for an array with shape (25, 14328947) and data type float64

In [3]:
behav_df = pd.read_csv(r'C:\Users\qmoreau\Documents\Work\Beta_bursts\Behavioral\behav_df_cleaned_new.csv')


In [4]:

def calculate_burst_rates(behav_df, df_burst_behav, epoch, group, time_bin=0.05, time_buffer=0.125):
    """
    Calculate burst rates for a given epoch, group, and time range.

    Parameters:
    - behav_df: DataFrame containing behavioral data
    - df_burst_behav: DataFrame containing burst behavioral data
    - epoch: The epoch for which burst rates are calculated
    - group: The group for which burst rates are calculated
    - time_bin: Time bin for burst rate calculation
    - time_buffer: Time buffer for extending the time range

    Returns:
    - burst_rates_dict: Dictionary containing burst rates for each subject, block, and coh_cat combination
    """
# Select subjects based on the group
    group_subjects = behav_df[behav_df['group'] == group]['subject'].unique()

    # Define the time range
    time_range = np.arange(-1 - time_buffer, 2 + time_buffer, time_bin)

    burst_df = df_burst_behav[df_burst_behav['epoch'] == epoch]

    # Create a dictionary to store burst rates for each subject, block, and coh_cat combination
    burst_rates_dict = {}

    for subject in group_subjects:
        # Initialize an array for burst rates for the current subject
        subject_burst_rates = np.zeros((len(time_range),))

        subject_df = burst_df[burst_df['subject'] == subject]

        # Group by block and coh_cat
        grouped_df = subject_df.groupby(['block', 'coh_cat'])

        for (block, coh_cat), group in grouped_df:
            # Calculate burst rates for the current subject
            for i, t in enumerate(time_range):
                subject_burst_rates[i] = len(group[(group['peak_time'] >= t) & (group['peak_time'] < t + time_bin)]) / time_bin

            subject_burst_rates_smoothed = gaussian_filter1d(subject_burst_rates, sigma=1)

            # baseline correction
            subject_burst_rates_smoothed = ((subject_burst_rates_smoothed - np.mean(subject_burst_rates_smoothed[12:19])) / np.mean(subject_burst_rates_smoothed[12:19])) * 100

            # Store the burst rates in the dictionary with subject, block, and coh_cat as keys
            key = (block, coh_cat)
            if key not in burst_rates_dict:
                burst_rates_dict[key] = []

            burst_rates_dict[key].append(subject_burst_rates_smoothed)


    # Convert the lists of arrays into a single NumPy array for each key
    for key in burst_rates_dict:
        burst_rates_dict[key] = np.array(burst_rates_dict[key])

    return burst_rates_dict

In [5]:

behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')


PC_list = ['PC_1']
# , 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'PC_7', 'PC_8', 'PC_9', 'PC_10',
#         'PC_11', 'PC_12', 'PC_13', 'PC_14', 'PC_15', 'PC_16', 'PC_17', 'PC_18', 'PC_19', 'PC_20']

for PC in PC_list:
    thresh=np.percentile(burst_df[PC], [25,50,75])
    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q0 = burst_df[(burst_df['peak_time'] >= 0.5) & (burst_df['peak_time'] <= 1.5) & (burst_df['epoch'] == 'vis') & (burst_df[PC] <= thresh[0])]

    # Group by subject and trial, and count the number of bursts
    burst_counts_Q0 = filtered_bursts_Q0.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q0 = behav_df.merge(burst_counts_Q0, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q1 = burst_df[(burst_df['peak_time'] >= 0.5) & (burst_df['peak_time'] <= 1.5) & (burst_df['epoch'] == 'vis') & (burst_df[PC] > thresh[0]) & (burst_df[PC] <= thresh[1])]

    # Group by subject and trial, and count the number of bursts
    burst_counts_Q1 = filtered_bursts_Q1.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q1 = behav_df.merge(burst_counts_Q1, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q2 = burst_df[(burst_df['peak_time'] >= 0.5) & (burst_df['peak_time'] <= 1.5) & (burst_df['epoch'] == 'vis') &  (burst_df[PC] > thresh[1]) & (burst_df[PC] <= thresh[2])]
    # Group by subject and trial, and count the number of bursts

    burst_counts_Q2 = filtered_bursts_Q2.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q2 = behav_df.merge(burst_counts_Q2, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q3 = burst_df[(burst_df['peak_time'] >= 0.5) & (burst_df['peak_time'] <= 1.5) & (burst_df['epoch'] == 'vis') & (burst_df[PC] > thresh[2])]
    # Group by subject and trial, and count the number of bursts

    burst_counts_Q3 = filtered_bursts_Q3.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q3 = behav_df.merge(burst_counts_Q3, on=['subject', 'block', 'trial'], how='left')

    behav_df_Q0.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q0.csv', index=False)
    behav_df_Q1.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q1.csv', index=False)
    behav_df_Q2.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q2.csv', index=False)
    behav_df_Q3.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q3.csv', index=False)



In [6]:

# Load behav_df
behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')

burst_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/burst_features.csv')

PC_list = ['PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'PC_7', 'PC_8', 'PC_9', 'PC_10',
        'PC_11', 'PC_12', 'PC_13', 'PC_14', 'PC_15', 'PC_16', 'PC_17', 'PC_18', 'PC_19', 'PC_20']

for PC in PC_list:
    thresh=np.percentile(burst_df[PC], [25,50,75])
    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q0 = burst_df[(burst_df['peak_time'] >= 0.25) & (burst_df['peak_time'] <= 0.75) & (burst_df['epoch'] == 'mot') & (burst_df[PC] <= thresh[0])]

    # Group by subject and trial, and count the number of bursts
    burst_counts_Q0 = filtered_bursts_Q0.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q0 = behav_df.merge(burst_counts_Q0, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q1 = burst_df[(burst_df['peak_time'] >= 0.25) & (burst_df['peak_time'] <= 0.75) & (burst_df['epoch'] == 'mot') & (burst_df[PC] > thresh[0]) & (burst_df[PC] <= thresh[1])]

    # Group by subject and trial, and count the number of bursts
    burst_counts_Q1 = filtered_bursts_Q1.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q1 = behav_df.merge(burst_counts_Q1, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q2 = burst_df[(burst_df['peak_time'] >= 0.25) & (burst_df['peak_time'] <= 0.75) & (burst_df['epoch'] == 'mot') &  (burst_df[PC] > thresh[1]) & (burst_df[PC] <= thresh[2])]
    # Group by subject and trial, and count the number of bursts

    burst_counts_Q2 = filtered_bursts_Q2.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q2 = behav_df.merge(burst_counts_Q2, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q3 = burst_df[(burst_df['peak_time'] >= 0.25) & (burst_df['peak_time'] <= 0.75) & (burst_df['epoch'] == 'mot') & (burst_df[PC] > thresh[2])]
    # Group by subject and trial, and count the number of bursts

    burst_counts_Q3 = filtered_bursts_Q3.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q3 = behav_df.merge(burst_counts_Q3, on=['subject', 'block', 'trial'], how='left')

    behav_df_Q0.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q0.csv', index=False)
    behav_df_Q1.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q1.csv', index=False)
    behav_df_Q2.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q2.csv', index=False)
    behav_df_Q3.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q3.csv', index=False)




OSError: Cannot save file into a non-existent directory: 'C:\Users\qmoreau\Documents\Work\Beta_bursts\Bursts\Motor_PCs\PC_1_Motor'

In [ ]:

behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')


PC_list = ['PC_7', 'PC_8', 'PC_9', 'PC_10']
# 'PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'PC_7', 'PC_8', 'PC_9', 'PC_10',
#         'PC_11', 'PC_12', 'PC_13', 'PC_14', 'PC_15', 'PC_16', 'PC_17', 'PC_18', 'PC_19', 'PC_20']

for PC in PC_list:
    thresh=np.percentile(burst_df[PC], [25,50,75])
    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q0 = burst_df[(burst_df['peak_time'] >= -1) & (burst_df['peak_time'] <= 0) & (burst_df['epoch'] == 'vis') & (burst_df[PC] <= thresh[0])]

    # Group by subject and trial, and count the number of bursts
    burst_counts_Q0 = filtered_bursts_Q0.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q0 = behav_df.merge(burst_counts_Q0, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q1 = burst_df[(burst_df['peak_time'] >= -1) & (burst_df['peak_time'] <= 0) & (burst_df['epoch'] == 'vis') & (burst_df[PC] > thresh[0]) & (burst_df[PC] <= thresh[1])]

    # Group by subject and trial, and count the number of bursts
    burst_counts_Q1 = filtered_bursts_Q1.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q1 = behav_df.merge(burst_counts_Q1, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q2 = burst_df[(burst_df['peak_time'] >= -1) & (burst_df['peak_time'] <= 0) & (burst_df['epoch'] == 'vis') &  (burst_df[PC] > thresh[1]) & (burst_df[PC] <= thresh[2])]
    # Group by subject and trial, and count the number of bursts

    burst_counts_Q2 = filtered_bursts_Q2.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q2 = behav_df.merge(burst_counts_Q2, on=['subject', 'block', 'trial'], how='left')

    # Filter bursts within the specified time range and epoch
    filtered_bursts_Q3 = burst_df[(burst_df['peak_time'] >= -1) & (burst_df['peak_time'] <= 0) & (burst_df['epoch'] == 'vis') & (burst_df[PC] > thresh[2])]
    # Group by subject and trial, and count the number of bursts

    burst_counts_Q3 = filtered_bursts_Q3.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

    # Merge burst counts with behav_df
    behav_df_Q3 = behav_df.merge(burst_counts_Q3, on=['subject', 'block', 'trial'], how='left')

    behav_df_Q0.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q0.csv', index=False)
    behav_df_Q1.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q1.csv', index=False)
    behav_df_Q2.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q2.csv', index=False)
    behav_df_Q3.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q3.csv', index=False)



In [ ]:
print(np.min(burst_df[burst_df['epoch'] == 'vis']['peak_time']))
print(np.max(burst_df[burst_df['epoch'] == 'vis']['peak_time']))
print(np.min(burst_df[burst_df['epoch'] == 'mot']['peak_time']))
print(np.max(burst_df[burst_df['epoch'] == 'mot']['peak_time']))


-0.87
1.8716666666666664
-0.87
1.3716666666666666


In [ ]:
# # Load your data
# behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')

# # Define the list of PCs
# PC_list = ['PC_7', 'PC_8', 'PC_9', 'PC_10']

# # Define sliding window parameters
# window_size = 0.2  # 200 ms in seconds
# step_size = 0.2    # Slide by 200 ms (no overlap)

# for PC in PC_list:
#     # Calculate the thresholds for quartiles
#     thresh = np.percentile(burst_df[PC], [25, 50, 75])
    
#     # Iterate over time windows
#     start_time = -1
#     end_time = 2
    
#     window_start = start_time
#     while window_start + window_size <= end_time:
#         window_end = window_start + window_size
        
#         # Process for each quartile
#         for q, (lower, upper) in enumerate(zip([None, thresh[0], thresh[1], thresh[2]], [thresh[0], thresh[1], thresh[2], None])):
            
#             # Define filters for quartiles
#             if lower is None:
#                 filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
#                                            (burst_df['peak_time'] < window_end) & 
#                                            (burst_df['epoch'] == 'vis') & 
#                                            (burst_df[PC] <= upper)]
#             elif upper is None:
#                 filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
#                                            (burst_df['peak_time'] < window_end) & 
#                                            (burst_df['epoch'] == 'vis') & 
#                                            (burst_df[PC] > lower)]
#             else:
#                 filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
#                                            (burst_df['peak_time'] < window_end) & 
#                                            (burst_df['epoch'] == 'vis') & 
#                                            (burst_df[PC] > lower) & 
#                                            (burst_df[PC] <= upper)]
            
#             # Group by subject and trial, count bursts in the window
#             burst_counts = filtered_bursts.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')
            
#             # Merge burst counts with behav_df
#             behav_df_window = behav_df.merge(burst_counts, on=['subject', 'block', 'trial'], how='left')
            
#             # Save the data to a CSV file
#             behav_df_window.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q{q}_window_{round(window_start, 1)}_{round(window_end, 1)}.csv', index=False)
        
#         # Slide the window forward by the step size
#         window_start += step_size


In [ ]:
# # Load your data
# behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')

# # Define the list of PCs
# PC_list = ['PC_7', 'PC_8', 'PC_9', 'PC_10']

# def clean_decimal(value, tolerance=1e-9):

#     if value == 1:  # Handle exact 1.0 cases
#         return 1
#     if value == 2:
#         return 2
#     elif isinstance(value, float):  # Handle float cases
#         rounded_value = round(value, 1)  # Round to the nearest tenth
#         return rounded_value if abs(rounded_value) >= tolerance else 0  # Handle near-zero values
#     elif value == -5.551115123125783e-17:
#         return 0
        

        
# # Define sliding window parameters
# window_size = 0.2  # 200 ms in seconds
# step_size = 0.1    # Slide by 200 ms (no overlap)

# for PC in PC_list:
#     # Calculate the thresholds for quartiles
#     thresh = np.percentile(burst_df[PC], [25, 50, 75])
    
#     # Iterate over time windows
#     start_time = -1
#     end_time = 2
    
#     window_start = start_time
#     while window_start + window_size <= end_time:
#         window_end = window_start + window_size
        
#         # Process for each quartile
#         for q, (lower, upper) in enumerate(zip([None, thresh[0], thresh[1], thresh[2]], [thresh[0], thresh[1], thresh[2], None])):
            
#             # Define filters for quartiles
#             if lower is None:
#                 filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
#                                            (burst_df['peak_time'] < window_end) & 
#                                            (burst_df['epoch'] == 'vis') & 
#                                            (burst_df[PC] <= upper)]
#             elif upper is None:
#                 filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
#                                            (burst_df['peak_time'] < window_end) & 
#                                            (burst_df['epoch'] == 'vis') & 
#                                            (burst_df[PC] > lower)]
#             else:
#                 filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
#                                            (burst_df['peak_time'] < window_end) & 
#                                            (burst_df['epoch'] == 'vis') & 
#                                            (burst_df[PC] > lower) & 
#                                            (burst_df[PC] <= upper)]
            
#             # Group by subject and trial, count bursts in the window
#             burst_counts = filtered_bursts.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')
            
#             # Merge burst counts with behav_df
#             behav_df_window = behav_df.merge(burst_counts, on=['subject', 'block', 'trial'], how='left')
            
#             behav_df_window.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q{q}_window_{clean_decimal(window_start)}_{clean_decimal(window_end)}.csv', index=False)
#             print(window_start)
#         # Slide the window forward by the step size
#         window_start += step_size


-1
-1
-1
-1
-0.9
-0.9
-0.9
-0.9
-0.8
-0.8
-0.8
-0.8
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.3999999999999999
0.3999999999999999
0.

In [ ]:
# Define sliding window parameters
window_size = 0.2  # 200 ms in seconds
step_size = 0.1    # Slide by 100 ms

for PC in PC_list:
    # Calculate the thresholds for quartiles
    thresh = np.percentile(burst_df[PC], [25, 50, 75])
    
    # Iterate over time windows
    start_time = -1
    end_time = 2
    
    window_start = start_time
    while window_start + window_size <= end_time:
        window_end = window_start + window_size
        
        # Process for each quartile
        for q, (lower, upper) in enumerate(zip([None, thresh[0], thresh[1], thresh[2]], [thresh[0], thresh[1], thresh[2], None])):
            
            # Define filters for quartiles
            if lower is None:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'vis') & 
                                           (burst_df[PC] <= upper)]
            elif upper is None:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'vis') & 
                                           (burst_df[PC] > lower)]
            else:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'vis') & 
                                           (burst_df[PC] > lower) & 
                                           (burst_df[PC] <= upper)]
            
            # Group by subject and trial, count bursts in the window
            burst_counts = filtered_bursts.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')
            
            # Merge burst counts with behav_df
            behav_df_window = behav_df.merge(burst_counts, on=['subject', 'block', 'trial'], how='left')
            
            # Use string formatting and handle -0 explicitly
            window_start_str = f"{window_start:.3f}".rstrip('0').rstrip('.')
            window_end_str = f"{window_end:.3f}".rstrip('0').rstrip('.')
            
            # Handle the case where small negative values might display as '-0'
            if window_start_str == '-0':
                window_start_str = '0'
            if window_end_str == '-0':
                window_end_str = '0'
            
            behav_df_window.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs_baseline/{PC}_Visual_BL/behav_df_Q{q}_window_{window_start_str}_{window_end_str}.csv', index=False)
            print(window_start)
        
        # Slide the window forward by the step size
        window_start += step_size


-1
-1
-1
-1
-0.9
-0.9
-0.9
-0.9
-0.8
-0.8
-0.8
-0.8
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.3999999999999999
0.3999999999999999
0.

In [ ]:
behav_df

,subject,group,block,trial,trial_coherence,trial_perturb,trial_type,reach_dur,reach_rt,trial_directions,...,reach_real_angle,true_target_angle,coh_cat,perturb_cat,reach_vis_angle,reach_vis_err,reach_vis_abs_err,aim_vis_angle,aim_vis_err,aim_vis_abs_err
0,sub-101,Explicit,0,0,0.0,0.0,0,0.058329,0.483496,0,...,36.243775,60.0,zero,0.0,36.243775,23.756225,23.756225,48.930009,11.069991,11.069991
1,sub-101,Explicit,0,1,0.0,0.0,0,0.049986,0.441427,0,...,22.398256,30.0,zero,0.0,22.398256,7.601744,7.601744,17.125191,12.874809,12.874809
2,sub-101,Explicit,0,2,0.0,0.0,0,0.049997,0.483497,0,...,76.963217,90.0,zero,0.0,76.963217,13.036783,13.036783,67.627467,22.372533,22.372533
3,sub-101,Explicit,0,3,0.0,0.0,0,0.050007,0.500159,0,...,80.695139,90.0,zero,0.0,80.695139,9.304861,9.304861,69.827323,20.172677,20.172677
4,sub-101,Explicit,0,4,0.0,0.0,0,0.050015,0.408428,0,...,16.098772,30.0,zero,0.0,16.098772,13.901228,13.901228,19.496198,10.503802,10.503802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16571,sub-145,Explicit,7,51,0.0,0.0,2,0.050015,0.591832,0,...,89.996736,90.0,zero,0.0,89.996736,0.003264,0.003264,95.546031,-5.546031,5.546031
16572,sub-145,Explicit,7,52,0.0,0.0,2,0.050008,0.291736,0,...,99.338096,90.0,zero,0.0,99.338096,-9.338096,9.338096,110.125616,-20.125616,20.125616
16573,sub-145,Explicit,7,53,0.0,0.0,2,0.050027,0.375074,0,...,93.263702,90.0,zero,0.0,93.263702,-3.263702,3.263702,96.966572,-6.966572,6.966572
16574,sub-145,Explicit,7,54,0.0,0.0,2,0.050010,0.400098,0,...,44.020157,30.0,zero,0.0,44.020157,-14.020157,14.020157,55.093923,-25.093923,25.093923


In [6]:
# Define sliding window parameters
window_size = 0.2  # 200 ms in seconds
step_size = 0.1    # Slide by 100 ms

PC_list_mot = ['PC_14']
for PC in PC_list_mot:
    # Calculate the thresholds for quartiles
    thresh = np.percentile(burst_df[PC], [25, 50, 75])
    
    # Iterate over time windows
    start_time = -1
    end_time = 1.5
    
    window_start = start_time
    while window_start + window_size <= end_time:
        window_end = window_start + window_size
        
        # Process for each quartile
        for q, (lower, upper) in enumerate(zip([None, thresh[0], thresh[1], thresh[2]], [thresh[0], thresh[1], thresh[2], None])):
            
            # Define filters for quartiles
            if lower is None:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'mot') & 
                                           (burst_df[PC] <= upper)]
            elif upper is None:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'mot') & 
                                           (burst_df[PC] > lower)]
            else:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'mot') & 
                                           (burst_df[PC] > lower) & 
                                           (burst_df[PC] <= upper)]
            
            # Group by subject and trial, count bursts in the window
            burst_counts = filtered_bursts.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')
            
            # Merge burst counts with behav_df
            behav_df_window = behav_df.merge(burst_counts, on=['subject', 'block', 'trial'], how='left')
            
            # Use string formatting and handle -0 explicitly
            window_start_str = f"{window_start:.3f}".rstrip('0').rstrip('.')
            window_end_str = f"{window_end:.3f}".rstrip('0').rstrip('.')
            
            # Handle the case where small negative values might display as '-0'
            if window_start_str == '-0':
                window_start_str = '0'
            if window_end_str == '-0':
                window_end_str = '0'
            
            behav_df_window.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs_baseline/{PC}_Motor_BL/behav_df_Q{q}_window_{window_start_str}_{window_end_str}.csv', index=False)
            print(window_start)
        
        # Slide the window forward by the step size
        window_start += step_size


-1
-1
-1
-1
-0.9
-0.9
-0.9
-0.9
-0.8
-0.8
-0.8
-0.8
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.3999999999999999
0.3999999999999999
0.

In [11]:
# Define sliding window parameters
window_size = 0.2  # 200 ms in seconds
step_size = 0.1    # Slide by 100 ms

PC_list_mot = ['PC_7']
for PC in PC_list_mot:
    # Calculate the thresholds for terthiles
    thresh = np.percentile(burst_df[PC], [33, 66])
    
    # Iterate over time windows
    start_time = -1
    end_time = 1.5
    
    window_start = start_time
    while window_start + window_size <= end_time:
        window_end = window_start + window_size
        
        # Process for each terthile with correct unpacking
        terthile_bounds = [
            (None, thresh[0]),
            (thresh[0], thresh[1]),
            (thresh[1], None)
        ]
        
        for q, (lower, upper) in enumerate(terthile_bounds):
            # Define filters for terthiles
            if lower is None:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'mot') & 
                                           (burst_df[PC] <= upper)]
            elif upper is None:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'mot') & 
                                           (burst_df[PC] > lower)]
            else:
                filtered_bursts = burst_df[(burst_df['peak_time'] >= window_start) & 
                                           (burst_df['peak_time'] < window_end) & 
                                           (burst_df['epoch'] == 'mot') & 
                                           (burst_df[PC] > lower) & 
                                           (burst_df[PC] <= upper)]
            
            # Group by subject and trial, count bursts in the window
            burst_counts = filtered_bursts.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')
            
            # Merge burst counts with behav_df
            behav_df_window = behav_df.merge(burst_counts, on=['subject', 'block', 'trial'], how='left')
            
            # Use string formatting and handle -0 explicitly
            window_start_str = f"{window_start:.3f}".rstrip('0').rstrip('.')
            window_end_str = f"{window_end:.3f}".rstrip('0').rstrip('.')
            
            # Handle the case where small negative values might display as '-0'
            if window_start_str == '-0':
                window_start_str = '0'
            if window_end_str == '-0':
                window_end_str = '0'
            
            behav_df_window.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs_baseline/{PC}_Motor_BL_ter/behav_df_T{q}_window_{window_start_str}_{window_end_str}.csv', index=False)
            print(window_start)
        
        # Slide the window forward by the step size
        window_start += step_size


-1
-1
-1
-0.9
-0.9
-0.9
-0.8
-0.8
-0.8
-0.7000000000000001
-0.7000000000000001
-0.7000000000000001
-0.6000000000000001
-0.6000000000000001
-0.6000000000000001
-0.5000000000000001
-0.5000000000000001
-0.5000000000000001
-0.40000000000000013
-0.40000000000000013
-0.40000000000000013
-0.30000000000000016
-0.30000000000000016
-0.30000000000000016
-0.20000000000000015
-0.20000000000000015
-0.20000000000000015
-0.10000000000000014
-0.10000000000000014
-0.10000000000000014
-1.3877787807814457e-16
-1.3877787807814457e-16
-1.3877787807814457e-16
0.09999999999999987
0.09999999999999987
0.09999999999999987
0.19999999999999987
0.19999999999999987
0.19999999999999987
0.2999999999999999
0.2999999999999999
0.2999999999999999
0.3999999999999999
0.3999999999999999
0.3999999999999999
0.4999999999999999
0.4999999999999999
0.4999999999999999
0.5999999999999999
0.5999999999999999
0.5999999999999999
0.6999999999999998
0.6999999999999998
0.6999999999999998
0.7999999999999998
0.7999999999999998
0.799999999999

In [ ]:

# # combine PCs
# # Load behav_df
# behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')

# burst_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/burst_features.csv')

# PC_list = ['PC_10', 'PC_11']


# thresh0=np.percentile(burst_df[PC_list[0]], [25,50,75])
# thresh1=np.percentile(burst_df[PC_list[1]], [25,50,75])
# # thresh2=np.percentile(burst_df[PC_list[2]], [25,50,75])
# # thresh3=np.percentile(burst_df[PC_list[3]], [25,50,75])
# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q0 = burst_df[(burst_df['peak_time'] >= 0.25) & 
#                                (burst_df['peak_time'] <= 0.75) & 
#                                (burst_df['epoch'] == 'mot') & 
#                                ((burst_df[PC_list[0]] <= thresh0[0]) | 
#                                 (burst_df[PC_list[1]] <= thresh1[0]))]
#                                 # |
#                                 # (burst_df[PC_list[2]] <= thresh2[0])| 
#                                 # (burst_df[PC_list[3]] <= thresh3[0]))]


# # Group by subject and trial, and count the number of bursts
# burst_counts_Q0 = filtered_bursts_Q0.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q0 = behav_df.merge(burst_counts_Q0, on=['subject', 'block', 'trial'], how='left')

# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q1 = burst_df[(burst_df['peak_time'] >= 0.25) & 
#                               (burst_df['peak_time'] <= 0.75) & 
#                               (burst_df['epoch'] == 'mot') & 
#                               (((burst_df[PC_list[0]] > thresh0[0]) & (burst_df[PC_list[0]]<= thresh0[1])) |
#                               (((burst_df[PC_list[1]] > thresh1[0]) & (burst_df[PC_list[1]]<= thresh1[1]))))]
                               
#                               #   |
#                               # (((burst_df[PC_list[2]] > thresh2[0]) & (burst_df[PC_list[2]]<= thresh2[1])) |
#                               # (((burst_df[PC_list[3]] > thresh3[0]) & (burst_df[PC_list[3]]<= thresh3[1]))))))]


# # Group by subject and trial, and count the number of bursts
# burst_counts_Q1 = filtered_bursts_Q1.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q1 = behav_df.merge(burst_counts_Q1, on=['subject', 'block', 'trial'], how='left')

# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q2 = burst_df[(burst_df['peak_time'] >= 0.25) &
#                                (burst_df['peak_time'] <= 0.75) &
#                                  (burst_df['epoch'] == 'mot') &
#                               (((burst_df[PC_list[0]] > thresh0[1]) & (burst_df[PC_list[0]]<= thresh0[2])) |
#                               (((burst_df[PC_list[1]] > thresh1[1]) & (burst_df[PC_list[1]]<= thresh1[2]))))]
#                               # |
#                               #                                (((burst_df[PC_list[2]] > thresh2[1]) & (burst_df[PC_list[2]]<= thresh2[2])) |
#                               # (((burst_df[PC_list[3]] > thresh3[1]) & (burst_df[PC_list[3]]<= thresh3[2]))))))]


# # Group by subject and trial, and count the number of bursts
# burst_counts_Q2 = filtered_bursts_Q2.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q2 = behav_df.merge(burst_counts_Q2, on=['subject', 'block', 'trial'], how='left')

# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q3 = burst_df[(burst_df['peak_time'] >= 0.25) &
#                                (burst_df['peak_time'] <= 0.75) &
#                                  (burst_df['epoch'] == 'mot') &
#                                     ((burst_df[PC_list[0]] > thresh0[2]) |
#                                     (burst_df[PC_list[1]] > thresh1[2]))]

#                                     # |
#                                     # (burst_df[PC_list[2]] > thresh2[2]) |
#                                     # (burst_df[PC_list[3]] > thresh3[2]))]

# # Group by subject and trial, and count the number of bursts
# burst_counts_Q3 = filtered_bursts_Q3.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q3 = behav_df.merge(burst_counts_Q3, on=['subject', 'block', 'trial'], how='left')

# PC = "combined_PC_10_11"
# behav_df_Q0.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q0.csv', index=False)
# behav_df_Q1.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q1.csv', index=False)
# behav_df_Q2.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q2.csv', index=False)
# behav_df_Q3.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/{PC}_Motor/behav_df_Q3.csv', index=False)




In [ ]:

# # combine PCs Visual epochs
# # Load behav_df
# behav_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Behavioral/behav_df_cleaned_new.csv')

# burst_df = pd.read_csv('C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/burst_features.csv')

# PC_list = ['PC_10', 'PC_11']


# thresh0=np.percentile(burst_df[PC_list[0]], [25,50,75])
# thresh1=np.percentile(burst_df[PC_list[1]], [25,50,75])
# # thresh2=np.percentile(burst_df[PC_list[2]], [25,50,75])
# # thresh3=np.percentile(burst_df[PC_list[3]], [25,50,75])
# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q0 = burst_df[(burst_df['peak_time'] >= 0.5) & 
#                                (burst_df['peak_time'] <= 1.5) & 
#                                (burst_df['epoch'] == 'vis') & 
#                                ((burst_df[PC_list[0]] <= thresh0[0]) | 
#                                 (burst_df[PC_list[1]] <= thresh1[0]))]
#                                 # |
#                                 # (burst_df[PC_list[2]] <= thresh2[0])| 
#                                 # (burst_df[PC_list[3]] <= thresh3[0]))]


# # Group by subject and trial, and count the number of bursts
# burst_counts_Q0 = filtered_bursts_Q0.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q0 = behav_df.merge(burst_counts_Q0, on=['subject', 'block', 'trial'], how='left')

# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q1 = burst_df[(burst_df['peak_time'] >= 0.5) & 
#                               (burst_df['peak_time'] <= 1.5) & 
#                               (burst_df['epoch'] == 'vis') & 
#                               (((burst_df[PC_list[0]] > thresh0[0]) & (burst_df[PC_list[0]]<= thresh0[1])) |
#                               (((burst_df[PC_list[1]] > thresh1[0]) & (burst_df[PC_list[1]]<= thresh1[1]))))]
                               
#                               #   |
#                               # (((burst_df[PC_list[2]] > thresh2[0]) & (burst_df[PC_list[2]]<= thresh2[1])) |
#                               # (((burst_df[PC_list[3]] > thresh3[0]) & (burst_df[PC_list[3]]<= thresh3[1]))))))]


# # Group by subject and trial, and count the number of bursts
# burst_counts_Q1 = filtered_bursts_Q1.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q1 = behav_df.merge(burst_counts_Q1, on=['subject', 'block', 'trial'], how='left')

# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q2 = burst_df[(burst_df['peak_time'] >= 0.5) &
#                                (burst_df['peak_time'] <= 1.5) &
#                                  (burst_df['epoch'] == 'vis') &
#                               (((burst_df[PC_list[0]] > thresh0[1]) & (burst_df[PC_list[0]]<= thresh0[2])) |
#                               (((burst_df[PC_list[1]] > thresh1[1]) & (burst_df[PC_list[1]]<= thresh1[2]))))]
#                               # |
#                               #                                (((burst_df[PC_list[2]] > thresh2[1]) & (burst_df[PC_list[2]]<= thresh2[2])) |
#                               # (((burst_df[PC_list[3]] > thresh3[1]) & (burst_df[PC_list[3]]<= thresh3[2]))))))]


# # Group by subject and trial, and count the number of bursts
# burst_counts_Q2 = filtered_bursts_Q2.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q2 = behav_df.merge(burst_counts_Q2, on=['subject', 'block', 'trial'], how='left')

# # Filter bursts within the specified time range and epoch
# filtered_bursts_Q3 = burst_df[(burst_df['peak_time'] >= 0.5) &
#                                (burst_df['peak_time'] <= 1.5) &
#                                  (burst_df['epoch'] == 'vis') &
#                                     ((burst_df[PC_list[0]] > thresh0[2]) |
#                                     (burst_df[PC_list[1]] > thresh1[2]))]

#                                     # |
#                                     # (burst_df[PC_list[2]] > thresh2[2]) |
#                                     # (burst_df[PC_list[3]] > thresh3[2]))]

# # Group by subject and trial, and count the number of bursts
# burst_counts_Q3 = filtered_bursts_Q3.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# # Merge burst counts with behav_df
# behav_df_Q3 = behav_df.merge(burst_counts_Q3, on=['subject', 'block', 'trial'], how='left')

# PC = "combined_PC_10_11"
# behav_df_Q0.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q0.csv', index=False)
# behav_df_Q1.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q1.csv', index=False)
# behav_df_Q2.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q2.csv', index=False)
# behav_df_Q3.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Visual_PCs/{PC}_Visual/behav_df_Q3.csv', index=False)




In [ ]:
# Filter bursts within the specified time range and epoch
overall_bursts = burst_df[(burst_df['peak_time'] >= 0.25) & 
                               (burst_df['peak_time'] <= 0.75) & 
                               (burst_df['epoch'] == 'mot')]

overall_bursts_count = overall_bursts.groupby(['subject', 'block', 'trial']).size().reset_index(name='burst_count')

# Merge burst counts with behav_df
behav_df_overall = behav_df.merge(overall_bursts_count, on=['subject', 'block', 'trial'], how='left')

behav_df_overall.to_csv(f'C:/Users/qmoreau/Documents/Work/Beta_bursts/Bursts/Motor_PCs/overall_bursts.csv', index=False)
